In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [77]:
import pandas as pd
from tqdm import tqdm_notebook
import torch
import torch.nn
import torch.nn.functional as F
import sys
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
from sith import SITH

import seaborn as sns
import matplotlib.pyplot as plt
# import matplotlib.patheffects
import seaborn as sn
sn.set_context("poster")

cuda


# Initialize "Corpus"

Setup the corpus, tokenize the input, and create a list of the target letters

In [79]:
sentence_enders = ["?", ".", "!"]
separators = ["\'", ","]

letters = list("abcdefghijklmnopqrstuvwxyz".upper()) + ["<COM>", "<END>", "<SPA>"]
let_to_id = {s:x for x,s in enumerate(letters)}
id_to_let = dict([[v,k] for k,v in let_to_id.items()])
full_text = open("plagueis.txt", "r").read()
full_text = full_text.upper()
full_text = full_text.replace("...", ".")
split_text = full_text.split()
master_list = []
id_list = []
for s in split_text:
    for l in s:
        if l in sentence_enders:
            master_list.append("<END>")
            id_list.append(let_to_id["<END>"])
        elif l in separators:
            master_list.append("<COM>")
            id_list.append(let_to_id["<COM>"])
        else:
            master_list.append(l)
            id_list.append(let_to_id[l])
    master_list.append("<SPA>")
    id_list.append(let_to_id["<SPA>"])

def tokenize(inp_list, num_tokens=10):
    output = torch.zeros(len(inp_list), num_tokens, 1).type(torch.DoubleTensor)
    for i, inp in enumerate(inp_list):
        output[i, inp, 0] = 1.0
    return output

input_tokens = tokenize(id_list[:-1], len(list(let_to_id.keys())))
#target_tokens = tokenize(id_list[1:], len(list(let_to_id.keys())))
target_tokens = torch.DoubleTensor(id_list[1:]).view(-1, 1, 1)
if torch.cuda.is_available():
    input_tokens = input_tokens.cuda()
    target_tokens = target_tokens.cuda()

tokens = torch.cat((input_tokens, target_tokens), 1)



# Test the SITH parameters

pic parameters here to be used further down in the model. The output taustars will be printed in the next cell. Change tau_0, k, c, and T_every to get the right taustars

In [80]:
sith_params ={"in_features":len(list(let_to_id.keys())),
              "tau_0":1, "k":4,
              "c":.05, "ntau":140, "dt":.1, 
              "T_every":8, "alpha":1.0}
# This is only for making sure you pick the right parameters
sithrep = SITH(**sith_params)
sithrep.cuda()
taustars = sithrep._tau_star[sith_params['k']:-sith_params['k']:sith_params['T_every']].detach().cpu().numpy()

print(taustars)

[  1.           1.47745544   2.18287459   3.22509994   4.76494147
   7.03998871  10.40126965  15.36741246  22.7046672   33.54513415
  49.56144107  73.22482091 108.18641027 159.8406008  236.15736578
 348.91198567 515.50191263 761.63110709]


# Define the model

Its a simple model. Takes tensor of size (batch, num_tokens) as input. Passes into SITH, then into a hidden layer. The output is passed through a log_softmax, and is of size (batch, num_tokens) predicting the next letter token. 

In [81]:
class LetterModel(torch.nn.Module):
    def __init__(self, sith_params, num_tokens):
        super(LetterModel, self).__init__()
        #self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.sith = SITH(**sith_params)
        self.sith.cuda()
        num_taustars = self.sith._tau_star[sith_params['k']:-sith_params['k']:sith_params['T_every']].shape[0]
        self.linear = torch.nn.Linear(num_tokens*num_taustars, num_tokens).double()
    
    def reset(self):
        self.sith.reset()
        
    def forward(self, inputs):
        batch_size = inputs.shape[0]
        x = F.relu(self.sith(inputs)).view(batch_size, -1)
        x = self.linear(x)
        log_probs = F.log_softmax(x, dim=-1)
        return log_probs


# Define and Train the Model

In [82]:
model = LetterModel(sith_params, len(list(let_to_id.keys()))).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=.001)
loss_func = torch.nn.NLLLoss()
input_scaling = 10

In [83]:
epochs = 1000
progress_bar = tqdm_notebook(range(int(epochs)))
for e in progress_bar:
    model.reset()

    for inp_and_tok in tokens.split(20, dim=0):
        inps = inp_and_tok[:, :-1, 0]
        targets = inp_and_tok[:, -1, 0].type(torch.cuda.LongTensor)
        model.zero_grad()
        out = model(inps*input_scaling)
        loss = loss_func(out, targets)
        loss.backward()
        optimizer.step()
    model.reset()
    t = 0
    correct = 0
    pp = 0
    for inp_and_tok in tokens.split(20, dim=0):
        inps = inp_and_tok[:, :-1, 0]
        targets = inp_and_tok[:, -1, 0].type(torch.cuda.LongTensor)
        out = model(inps*input_scaling)

        # Accuracy
        correct += (out.argmax(-1) == targets).sum().detach().cpu().numpy()
        t += inps.shape[0]

    acc = correct/t
    progress_bar.set_description("%i: Acc: %0.4f" % (e, acc))

A Jupyter Widget

# Test the finished model

Prints out the most likely letter to occur when taking in the last letter as input. This cell is just a proof of concept that the model is working at all. 

This cell DOES NOT pass in the last predicted letter as inp

In [85]:
model.reset()
t = 0
correct = 0
pp = 0
for inp_and_tok in tokens.split(20, dim=0):
    inps = inp_and_tok[:, :-1, 0]
    targets = inp_and_tok[:, -1, 0].type(torch.cuda.LongTensor)
    out = model(inps*input_scaling)

    # Accuracy
    indexes = out.argmax(-1).detach().cpu().numpy()
    for i in indexes:
        sys.stdout.write(id_to_let[i])


ID<SPA>YOU<SPA>EVER<SPA>HEARETHE<SPA>TRAGEDY<SPA>OF<SPA>DARTH<SPA>PLAGUEIS<SPA>THE<SPA>WISE<END><SPA>I<SPA>THOUGHT<SPA>NOT<END><SPA>IT<COM>S<SPA>NOT<SPA>AESOTRY<SPA>THE<SPA>JEDI<SPA>WOULD<SPA>TELL<SPA>YOU<END><SPA>IT<COM>S<SPA>A<SPA>SITH<SPA>LEGEND<END><SPA>DARTH<SPA>PLAGUEIS<SPA>WAS<SPA>A<SPA>DARK<SPA>LORD<SPA>OF<SPA>THE<SPA>SITH<COM><SPA>SO<SPA>POWERFUL<SPA>AND<SPA>SO<SPA>WISE<SPA>HE<SPA>COULD<SPA>USE<SPA>THE<SPA>FORCE<SPA>TOEINFLUENCE<SPA>THE<SPA>MIDICHLORIANS<SPA>TOECREATE<SPA>LIFE<END>EHE<SPA>HAD<SPA>SUCH<SPA>AAKNOWLEDGE<SPA>OF<SPA>THE<SPA>DARK<SPA>SIDE<SPA>THAT<SPA>HE<SPA>COULDEEVEN<SPA>KEEP<SPA>THE<SPA>ONES<SPA>HE<SPA>CARED<SPA>ABOUT<SPA>FROM<SPA>DYING<END><SPA>THE<SPA>DARK<SPA>SIDE<SPA>OF<SPA>THE<SPA>FORCE<SPA>IS<SPA>A<SPA>PATHWAY<SPA>TO<SPA>MANY<SPA>ABILITIEESSOME<SPA>CONSIDERTTOEBE<SPA>UNNATURAL<END><SPA>HE<SPA>BECAME<SPA>SO<SPA>POWERFUL<END><SPA>THE<SPA>ONLY<SPA>THING<SPA>HE<SPA>WAS<SPA>AFRAID<SPA>OF<SPA>WAS<SPA>LOSING<SPA>HIS<SPA>POWER<COM><SPA>WHICHHEVENTUALLY<COM><SPA>OF<